In [1]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score, roc_auc_score

# from pykospacing import Spacing
# import re

import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
train_df = pd.read_csv(r'./data/korean-hate-speech-detection/train.hate.csv')

train_text = open(r'./data/korean-hate-speech-detection/train.news_title.txt', 'r')
train_texts = train_text.readlines()

train_text_list = []
for line in train_texts:
    train_text_list.append(line)
train_text.close()

train_df['text'] = train_text_list
train_df

In [ ]:
lengh = []
for i in train_df['comments']:
    lengh.append(len(i))
max(lengh)

In [54]:
spacing = Spacing()

def preprocessing(text):
    text = spacing(text)
    text = text.lower()  # 소문자 변경
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 기본 불용어 불러오기
korean_stopwords_path = "data/stopwords-ko.txt"
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

# 불용어 처리 함수 수정
def remove_stopwords(text, stopwords):
    words = text.split()  # 문장을 단어로 분리
    filtered_text = [word for word in words if word not in stopwords]  # 단어가 불용어 목록에 없는 경우만 추가
    return ' '.join(filtered_text)  # 필터링된 단어들 다시 조합

In [ ]:
# 문장 전처리 및 불용어 처리
for i in tqdm(range(len(train_df))):
    comment_text = train_df.loc[i, 'comments']
    newstile_text = train_df.loc[i, 'text']
    processed_comment = preprocessing(comment_text)
    processed_newstile = preprocessing(newstile_text)
    cleaned_comment = remove_stopwords(processed_comment, stopwords)
    cleaned_newstile = remove_stopwords(processed_newstile, stopwords)
    train_df.loc[i, 'processed_comments'] = cleaned_comment
    train_df.loc[i, 'processed_newstitle'] = cleaned_newstile

In [ ]:
train_df.loc[train_df['label'] == 'none', 'labels'] = 0
train_df.loc[train_df['label'] == 'offensive', 'labels'] = 1
train_df.loc[train_df['label'] == 'hate', 'labels'] = 2

train_df['labels'] = train_df['labels'].astype(int)
train_df.info()

In [70]:
train_df.to_csv('./data/train_df_processed.csv', index=False, encoding='utf-8-sig')

In [ ]:
dev_df = pd.read_csv(r'./data/korean-hate-speech-detection/dev.hate.csv')

dev_text = open(r'./data/korean-hate-speech-detection/dev.news_title.txt', 'r')
dev_texts = dev_text.readlines()

dev_text_list = []
for line in dev_texts:
    dev_text_list.append(line)
dev_text.close

dev_df['text'] = dev_text_list
dev_df

In [ ]:
# 문장 전처리 및 불용어 처리
for i in tqdm(range(len(dev_df))):
    comment_text = dev_df.loc[i, 'comments']
    newstile_text = dev_df.loc[i, 'text']
    processed_comment = preprocessing(comment_text)
    processed_newstile = preprocessing(newstile_text)
    cleaned_comment = remove_stopwords(processed_comment, stopwords)
    cleaned_newstile = remove_stopwords(processed_newstile, stopwords)
    dev_df.loc[i, 'processed_comments'] = cleaned_comment
    dev_df.loc[i, 'processed_newstitle'] = cleaned_newstile

In [88]:
dev_df.loc[dev_df['label'] == 'none', 'labels'] = 0
dev_df.loc[dev_df['label'] == 'offensive', 'labels'] = 1
dev_df.loc[dev_df['label'] == 'hate', 'labels'] = 2
dev_df['labels'] = dev_df['labels'].astype(int)

dev_df.to_csv('./data/dev_df_processed.csv', index=False, encoding='utf-8-sig')

In [ ]:
dev_df = pd.read_csv('./data/dev_df_processed.csv')
dev_df

In [ ]:
test_df = pd.read_csv(r'./data/korean-hate-speech-detection/test.hate.no_label.csv')

test_text = open(r'./data/korean-hate-speech-detection/test.news_title.txt')
test_texts = test_text.readlines()
test_text_list = []
for line in test_texts:
    test_text_list.append(line)

test_df['text'] = test_text_list
test_df

In [ ]:
# 문장 전처리 및 불용어 처리
for i in tqdm(range(len(test_df))):
    comment_text = test_df.loc[i, 'comments']
    newstile_text = test_df.loc[i, 'text']
    processed_comment = preprocessing(comment_text)
    processed_newstile = preprocessing(newstile_text)
    cleaned_comment = remove_stopwords(processed_comment, stopwords)
    cleaned_newstile = remove_stopwords(processed_newstile, stopwords)
    test_df.loc[i, 'processed_comments'] = cleaned_comment
    test_df.loc[i, 'processed_newstitle'] = cleaned_newstile

In [67]:
test_df.to_csv('./data/test_df_processed.csv', index=False, encoding='utf-8-sig')

In [23]:
train_df = pd.read_csv(r'./data/train_df_processed.csv')
dev_df = pd.read_csv(r'./data/dev_df_processed.csv')
test_df = pd.read_csv(r'./data/test_df_processed.csv')

In [3]:
model_name = 'beomi/KcELECTRA-base-v2022'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/p

In [32]:
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

with tf.device('/GPU:0'):
    tensor = tf.constant([1.0, 2.0, 3.0])
print("device:", tensor.device)

device: /job:localhost/replica:0/task:0/device:GPU:0


2024-10-27 13:47:10.513057: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-10-27 13:47:10.513437: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-10-27 13:47:10.513442: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-10-27 13:47:10.513741: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-27 13:47:10.514102: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [25]:
train_df = train_df.dropna()

,comments,label,text,processed_comments,processed_newstitle,labels
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate,"""밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]""\n",현재 호텔 주인 심정 아18 난 마른 하늘에 날 벼락 맞고 호텔 망하게 생겼는데 누...,밤새 조문 행렬 故 전미선 동료들이 그리워하는 따뜻한 배우 종합,2.0
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none,"""'연중' 故 전미선, 생전 마지막 미공개 인터뷰…환하게 웃는 모습 '먹먹'[종합]""\n",한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습그 모습 뒤의 슬픔을 미 처 ...,연중 故 전미선 생전 마지막 미공개 인터뷰 환하게 웃는 모습 먹먹종합,0.0
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate,"""[단독] 잔나비, 라디오 출연 취소→'한밤' 방송 연기..비판 여론 ing(종합)""\n",못된 넘들남의 고통을 즐겼던 넘들이젠 마땅한 처벌을 받아야지 그래야 공정한 사회지심...,단독 잔나비 라디오 출연 취소한밤 방송 연기비판 여론 ing종합,2.0
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none,"""'아스달 연대기' 장동건-김옥빈, 들끓는 '욕망커플'→눈물범벅 '칼끝 대립'""\n",2화 어설 펐는데 4화 지나서부터는 갈수록 너무 재밌던 데,아스달 연대기 장동건김옥빈 들끓는 욕망 커플 눈물범벅 칼끝 대립,0.0
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate,[DA:이슈] ‘구하라 비보’ 최종범 항소심에 영향?…법조계 “‘공소권 없음’ 아냐”\n,사람 얼굴 손톱으로 긁은 것은 인격 살해이고 동영상이 몰카냐 메걸리안 생각이 없노,da 이슈 구하라 비보 최종 범 항소심에 영향 법조계 공소권 없음 아냐,2.0
...,...,...,...,...,...,...
7891,힘내세요~ 응원합니다!!,none,"""허지웅, 허투루 넘길 말 없었다…솔직하게 드러냈던 속사정""\n",힘내세요 응원합니다,허지웅 허투루 넘길 말 없었다 솔직하게 드러냈던 속 사정,0.0
7892,힘내세요~~삼가 고인의 명복을 빕니다..,none,"""이혜경, ‘오! 캐롤’ 공연 중 남편 오정욱 부고…오열 속 발인 [종합]""\n",힘내세요삼가 고인의 명복을 빕니다,이혜경 캐롤 공연 중 남편 오정욱 부고오열 속 발인 종합,0.0
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,none,"""'설경구♥' 송윤아, 아들과 즐거운 하루 """"전 엄마니까요""""""\n",힘내세용 항상 응원합니닷,설경구 송윤아 아들과 즐거운 하루 전 엄마니까요,0.0
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,none,"""[SC현장]""""연예인 인생 협박 유감""""…미소잃은 최민수, '보복운전 혐의' 2차...",힘내소연기로 답해요 나도 53살 인데 저런 다 있더라구요인 격을 믿습니다홨팅,sc현장연예인 인생 협박 유감 미소 잃은 최민수 보복운전 혐의 2차 공판종합,0.0


In [33]:
# 데이터 토크나이징

tokenized_train_comment = tokenizer(
    list(train_df['processed_comments']),
    return_tensors='tf',
    max_length=128,
    padding=True,
    truncation=True,          # max_length 초과 토큰 truncate
    add_special_tokens=True,  # special token 추가
)

tokenized_train_title = tokenizer(
    list(train_df['processed_newstitle']),
    return_tensors='tf',
    max_length=128,
    padding=True,
    truncation=True,          # max_length 초과 토큰 truncate
    add_special_tokens=True,  # special token 추가
)

tokenized_dev_comment = tokenizer(
    list(dev_df['processed_comments']),
    return_tensors='tf',
    max_length=128,
    padding=True,
    truncation=True,          # max_length 초과 토큰 truncate
    add_special_tokens=True,  # special token 추가
)

tokenized_dev_title = tokenizer(
    list(dev_df['processed_newstitle']),
    return_tensors='tf',
    max_length=128,
    padding=True,
    truncation=True,          # max_length 초과 토큰 truncate
    add_special_tokens=True,  # special token 추가
)

In [39]:
# CLS 토큰 벡터 추출
train_comment_embedding = tokenized_train_comment['input_ids'][:, 0]
train_title_embedding = tokenized_train_title['input_ids'][:, 0]
dev_comment_embedding = tokenized_dev_comment['input_ids'][:, 0]
dev_title_embedding = tokenized_dev_title['input_ids'][:, 0]

In [45]:
train_df['labels'] = train_df['labels'].astype(int)

In [55]:
# 2D 배열로 변환
X_train = tf.reshape(train_comment_embedding, (-1, 1))
X_test = tf.reshape(dev_comment_embedding, (-1, 1))     # 테스트 데이터도 2D로 변환
y_train = train_df['labels']
y_test = dev_df['labels']

In [54]:
# 로지스틱 회귀 모델 학습
logistic_model = LogisticRegression(max_iter=1000, multi_class='multinomial')
logistic_model.fit(X_train, train_df['labels'])

# 예측 및 평가
y_pred = logistic_model.predict(X_test)
accuracy = accuracy_score(dev_df['labels'], y_pred)
f1 = f1_score(dev_df['labels'], y_pred, average='weighted')  # 다중 클래스 분류의 경우 'weighted' 또는 'macro' 사용
print("Accuracy:", accuracy)
print("F1 score:", f1)

Accuracy: 0.33970276008492567
F1 score: 0.1722739829273791


In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')  # 다중 클래스 분류의 경우 'weighted' 또는 'macro' 사용
print("Accuracy:", accuracy)
print("F1 score:", f1)

Accuracy: 0.33970276008492567
F1 score: 0.1722739829273791


In [65]:
X_test

<tf.Tensor: shape=(471, 1), dtype=int32, numpy=
array([[2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
    

In [64]:
rf_model.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [62]:
y_test

0      0
1      1
2      2
3      2
4      1
      ..
466    1
467    2
468    1
469    2
470    0
Name: labels, Length: 471, dtype: int64